## Import Dependencies

In [1]:
# Import our dependencies
import pandas as pd
import numpy as np
from pathlib import Path
from collections import Counter

In [2]:
import warnings
warnings.filterwarnings('ignore')

## Import Datasets

In [3]:
# Import dataset - credit_record.csv
credit_record_df = pd.read_csv('../Resources/credit_record.csv')
# Import dataset - application_record.csv
application_record_df = pd.read_csv('../Resources/application_record.csv')

## Datasets Demographics

In [4]:
# Dataframes display

credit_record_df.head(5)

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C


In [5]:
# Dataframes display

application_record_df.head(5)

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


In [6]:
# Get the info of the Dataframe - credit_record_df
credit_record_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 3 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   ID              1048575 non-null  int64 
 1   MONTHS_BALANCE  1048575 non-null  int64 
 2   STATUS          1048575 non-null  object
dtypes: int64(2), object(1)
memory usage: 24.0+ MB


In [7]:
# Get the info of the DataFrame - application_record_df
application_record_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438557 entries, 0 to 438556
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   ID                   438557 non-null  int64  
 1   CODE_GENDER          438557 non-null  object 
 2   FLAG_OWN_CAR         438557 non-null  object 
 3   FLAG_OWN_REALTY      438557 non-null  object 
 4   CNT_CHILDREN         438557 non-null  int64  
 5   AMT_INCOME_TOTAL     438557 non-null  float64
 6   NAME_INCOME_TYPE     438557 non-null  object 
 7   NAME_EDUCATION_TYPE  438557 non-null  object 
 8   NAME_FAMILY_STATUS   438557 non-null  object 
 9   NAME_HOUSING_TYPE    438557 non-null  object 
 10  DAYS_BIRTH           438557 non-null  int64  
 11  DAYS_EMPLOYED        438557 non-null  int64  
 12  FLAG_MOBIL           438557 non-null  int64  
 13  FLAG_WORK_PHONE      438557 non-null  int64  
 14  FLAG_PHONE           438557 non-null  int64  
 15  FLAG_EMAIL       

### Get the unique counts of each variable from the two DataFrames

In [8]:
credit_record_df.nunique()


ID                45985
MONTHS_BALANCE       61
STATUS                8
dtype: int64

In [9]:
application_record_df.nunique()

ID                     438510
CODE_GENDER                 2
FLAG_OWN_CAR                2
FLAG_OWN_REALTY             2
CNT_CHILDREN               12
AMT_INCOME_TOTAL          866
NAME_INCOME_TYPE            5
NAME_EDUCATION_TYPE         5
NAME_FAMILY_STATUS          5
NAME_HOUSING_TYPE           6
DAYS_BIRTH              16379
DAYS_EMPLOYED            9406
FLAG_MOBIL                  1
FLAG_WORK_PHONE             2
FLAG_PHONE                  2
FLAG_EMAIL                  2
OCCUPATION_TYPE            18
CNT_FAM_MEMBERS            13
dtype: int64

# Cleaning "credit_record_df"

In [10]:
# Count how many entries for each STATUS
credit_record_df['STATUS'].value_counts()

C    442031
0    383120
X    209230
1     11090
5      1693
2       868
3       320
4       223
Name: STATUS, dtype: int64

In [11]:
#Calculate the age of each MONTH_BALANCE & 
credit_record_df['Months_from_Today']= credit_record_df['MONTHS_BALANCE']* -1

In [12]:
# Sort by ID and Months_from_Today
credit_record_df = credit_record_df.sort_values(['ID','Months_from_Today'], ascending=True)
credit_record_df

,ID,MONTHS_BALANCE,STATUS,Months_from_Today
0,5001711,0,X,0
1,5001711,-1,0,1
2,5001711,-2,0,2
3,5001711,-3,0,3
4,5001712,0,C,0
...,...,...,...,...
1048570,5150487,-25,C,25
1048571,5150487,-26,C,26
1048572,5150487,-27,C,27
1048573,5150487,-28,C,28


### From above, oversampling will be an issue as over 99% of the data shows "bad" applicants

# Lucas' Idea of "Good" and "Bad" applicants
## First, we replace X and C to be 0 and 0-5 to be 1. The main idea here is count the number of bad months each participant has and if a participant has 3 or more "Bad" months (three 1's) then the participant is "Bad" and is then rejected for a credit card.



In [13]:
credit_record_df["STATUS"].replace({"X":0, "C":0, "0":1, "1":1, "2":1, "3":1, "4":1, "5":1} , inplace= True )

In [14]:
credit_record_df

,ID,MONTHS_BALANCE,STATUS,Months_from_Today
0,5001711,0,0,0
1,5001711,-1,1,1
2,5001711,-2,1,2
3,5001711,-3,1,3
4,5001712,0,0,0
...,...,...,...,...
1048570,5150487,-25,0,25
1048571,5150487,-26,0,26
1048572,5150487,-27,0,27
1048573,5150487,-28,0,28


In [15]:
lucas_df = credit_record_df
bad_months = lucas_df.groupby(["ID"])["STATUS"].sum()
bad_months

ID
5001711     3
5001712    10
5001713     0
5001714     0
5001715     0
           ..
5150482    12
5150483     0
5150484    12
5150485     2
5150487     0
Name: STATUS, Length: 45985, dtype: int64

In [16]:
status = bad_months.apply(lambda x: 0 if x >= 3 else 1)

In [17]:
new_credit_record = credit_record_df.merge(status, how="left", on="ID")
new_credit_record

,ID,MONTHS_BALANCE,STATUS_x,Months_from_Today,STATUS_y
0,5001711,0,0,0,0
1,5001711,-1,1,1,0
2,5001711,-2,1,2,0
3,5001711,-3,1,3,0
4,5001712,0,0,0,0
...,...,...,...,...,...
1048570,5150487,-25,0,25,1
1048571,5150487,-26,0,26,1
1048572,5150487,-27,0,27,1
1048573,5150487,-28,0,28,1


In [18]:
# Group by ID 
new_credit_grouped = new_credit_record.drop(["STATUS_x"], axis= 1)
new_credit_grouped

,ID,MONTHS_BALANCE,Months_from_Today,STATUS_y
0,5001711,0,0,0
1,5001711,-1,1,0
2,5001711,-2,2,0
3,5001711,-3,3,0
4,5001712,0,0,0
...,...,...,...,...
1048570,5150487,-25,25,1
1048571,5150487,-26,26,1
1048572,5150487,-27,27,1
1048573,5150487,-28,28,1


In [19]:
new_credit = new_credit_grouped.groupby("ID", as_index=False)
new_credit = new_credit.first()
new_credit

,ID,MONTHS_BALANCE,Months_from_Today,STATUS_y
0,5001711,0,0,0
1,5001712,0,0,0
2,5001713,0,0,1
3,5001714,0,0,1
4,5001715,0,0,1
...,...,...,...,...
45980,5150482,-11,11,0
45981,5150483,0,0,1
45982,5150484,0,0,0
45983,5150485,0,0,1


# Cleaning "application_record_df"

## Removing duplicate ID's

In [20]:
# Find number of duplicates in 'ID' column
len(application_record_df['ID']) - len(application_record_df['ID'].unique())

47

In [21]:
# Drop the duplicate entries from the 'ID' Column

application_record_df = application_record_df.drop_duplicates('ID', keep='last')

In [22]:
application_record_df.shape

(438510, 18)

## Finding the Null columns

In [23]:
new_credit.isnull().sum()

ID                   0
MONTHS_BALANCE       0
Months_from_Today    0
STATUS_y             0
dtype: int64

In [24]:
application_record_df.isnull().sum()

ID                          0
CODE_GENDER                 0
FLAG_OWN_CAR                0
FLAG_OWN_REALTY             0
CNT_CHILDREN                0
AMT_INCOME_TOTAL            0
NAME_INCOME_TYPE            0
NAME_EDUCATION_TYPE         0
NAME_FAMILY_STATUS          0
NAME_HOUSING_TYPE           0
DAYS_BIRTH                  0
DAYS_EMPLOYED               0
FLAG_MOBIL                  0
FLAG_WORK_PHONE             0
FLAG_PHONE                  0
FLAG_EMAIL                  0
OCCUPATION_TYPE        134187
CNT_FAM_MEMBERS             0
dtype: int64

In [25]:
# Replace nulls with "No Occupation Type"
application_record_df["OCCUPATION_TYPE"].fillna("No Occupation Type", inplace = True)
application_record_df.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,No Occupation Type,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,No Occupation Type,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


In [26]:
application_record_df.isnull().sum()

ID                     0
CODE_GENDER            0
FLAG_OWN_CAR           0
FLAG_OWN_REALTY        0
CNT_CHILDREN           0
AMT_INCOME_TOTAL       0
NAME_INCOME_TYPE       0
NAME_EDUCATION_TYPE    0
NAME_FAMILY_STATUS     0
NAME_HOUSING_TYPE      0
DAYS_BIRTH             0
DAYS_EMPLOYED          0
FLAG_MOBIL             0
FLAG_WORK_PHONE        0
FLAG_PHONE             0
FLAG_EMAIL             0
OCCUPATION_TYPE        0
CNT_FAM_MEMBERS        0
dtype: int64

## Convert string columns to numbers

In [27]:
#Convert string columns to numbers using get_dummies
application_record_df = pd.get_dummies(application_record_df, columns=["CODE_GENDER","FLAG_OWN_CAR","FLAG_OWN_REALTY",
                                  "NAME_INCOME_TYPE","NAME_EDUCATION_TYPE","NAME_FAMILY_STATUS",
                                  "NAME_HOUSING_TYPE","OCCUPATION_TYPE"], drop_first = True)

In [28]:
application_record_df.head()

,ID,CNT_CHILDREN,AMT_INCOME_TOTAL,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,...,OCCUPATION_TYPE_Low-skill Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_No Occupation Type,OCCUPATION_TYPE_Private service staff,OCCUPATION_TYPE_Realty agents,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Secretaries,OCCUPATION_TYPE_Security staff,OCCUPATION_TYPE_Waiters/barmen staff
0,5008804,0,427500.0,-12005,-4542,1,1,0,0,2.0,...,0,0,0,1,0,0,0,0,0,0
1,5008805,0,427500.0,-12005,-4542,1,1,0,0,2.0,...,0,0,0,1,0,0,0,0,0,0
2,5008806,0,112500.0,-21474,-1134,1,0,0,0,2.0,...,0,0,0,0,0,0,0,0,1,0
3,5008808,0,270000.0,-19110,-3051,1,0,1,1,1.0,...,0,0,0,0,0,0,1,0,0,0
4,5008809,0,270000.0,-19110,-3051,1,0,1,1,1.0,...,0,0,0,0,0,0,1,0,0,0


## Convert DAYS_BIRTH & DAYS_EMPLOYED into years

In [29]:
application_record_df['AGE'] = round(application_record_df['DAYS_BIRTH'] / -365, 2)

In [30]:
application_record_df['EMPLOYMENT_PERIOD'] = round(application_record_df['DAYS_EMPLOYED'] / -365, 2)

In [31]:
application_record_df.head(5)

,ID,CNT_CHILDREN,AMT_INCOME_TOTAL,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,...,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_No Occupation Type,OCCUPATION_TYPE_Private service staff,OCCUPATION_TYPE_Realty agents,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Secretaries,OCCUPATION_TYPE_Security staff,OCCUPATION_TYPE_Waiters/barmen staff,AGE,EMPLOYMENT_PERIOD
0,5008804,0,427500.0,-12005,-4542,1,1,0,0,2.0,...,0,1,0,0,0,0,0,0,32.89,12.44
1,5008805,0,427500.0,-12005,-4542,1,1,0,0,2.0,...,0,1,0,0,0,0,0,0,32.89,12.44
2,5008806,0,112500.0,-21474,-1134,1,0,0,0,2.0,...,0,0,0,0,0,0,1,0,58.83,3.11
3,5008808,0,270000.0,-19110,-3051,1,0,1,1,1.0,...,0,0,0,0,1,0,0,0,52.36,8.36
4,5008809,0,270000.0,-19110,-3051,1,0,1,1,1.0,...,0,0,0,0,1,0,0,0,52.36,8.36


## Create bins for continous variable categories column 'AMT_INCOME_TOTAL'

In [32]:
application_record_df.AMT_INCOME_TOTAL.min()

26100.0

In [33]:
application_record_df.AMT_INCOME_TOTAL.max()

6750000.0

In [34]:
# Creating bins for income amount

# bins = [0,25000,50000,75000,100000,125000,150000,175000,200000,225000,250000,275000,300000,325000,350000,375000,400000,425000,450000,475000,500000,10000000000]
# slot = ['0-25000', '25000-50000','50000-75000','75000,100000','100000-125000', '125000-150000', '150000-175000','175000-200000',
#        '200000-225000','225000-250000','250000-275000','275000-300000','300000-325000','325000-350000','350000-375000',
#        '375000-400000','400000-425000','425000-450000','450000-475000','475000-500000','500000 and above']

# application_record_df['AMT_INCOME_RANGE']=pd.cut(application_record_df['AMT_INCOME_TOTAL'],bins,labels=slot)

# Lucas
# The scale for Annual Income is much larger than all the other values in the dataset. We format by dividing by 100000 to rescale those data points
application_record_df['AMT_INCOME_TOTAL'] = application_record_df['AMT_INCOME_TOTAL'] / 100000

In [35]:
# application_record_df['AGE'] = application_record_df['AGE'] / 10

## Remove unnecessary columns from application_record

In [36]:
# Remove the columns that are not needed from the application_record_df

Columns_to_remove = ['DAYS_BIRTH', 'DAYS_EMPLOYED','FLAG_MOBIL','FLAG_WORK_PHONE','FLAG_PHONE','FLAG_EMAIL']

application_record_df.drop(labels=Columns_to_remove,axis=1,inplace=True)

In [37]:
application_record_df.head(5)

,ID,CNT_CHILDREN,AMT_INCOME_TOTAL,CNT_FAM_MEMBERS,CODE_GENDER_M,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_Y,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,...,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_No Occupation Type,OCCUPATION_TYPE_Private service staff,OCCUPATION_TYPE_Realty agents,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Secretaries,OCCUPATION_TYPE_Security staff,OCCUPATION_TYPE_Waiters/barmen staff,AGE,EMPLOYMENT_PERIOD
0,5008804,0,4.275,2.0,1,1,1,0,0,0,...,0,1,0,0,0,0,0,0,32.89,12.44
1,5008805,0,4.275,2.0,1,1,1,0,0,0,...,0,1,0,0,0,0,0,0,32.89,12.44
2,5008806,0,1.125,2.0,1,1,1,0,0,0,...,0,0,0,0,0,0,1,0,58.83,3.11
3,5008808,0,2.700,1.0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,52.36,8.36
4,5008809,0,2.700,1.0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,52.36,8.36


In [38]:
application_record_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 438510 entries, 0 to 438556
Data columns (total 44 columns):
 #   Column                                             Non-Null Count   Dtype  
---  ------                                             --------------   -----  
 0   ID                                                 438510 non-null  int64  
 1   CNT_CHILDREN                                       438510 non-null  int64  
 2   AMT_INCOME_TOTAL                                   438510 non-null  float64
 3   CNT_FAM_MEMBERS                                    438510 non-null  float64
 4   CODE_GENDER_M                                      438510 non-null  uint8  
 5   FLAG_OWN_CAR_Y                                     438510 non-null  uint8  
 6   FLAG_OWN_REALTY_Y                                  438510 non-null  uint8  
 7   NAME_INCOME_TYPE_Pensioner                         438510 non-null  uint8  
 8   NAME_INCOME_TYPE_State servant                     438510 non-null  uint8 

### Join the credit_record_grouped_df to the application_record_df

In [39]:
credit_application_df = application_record_df.join(new_credit.set_index('ID'), on='ID', how='inner')
credit_application_df.drop(['MONTHS_BALANCE'], axis=1, inplace=True)

credit_application_df

,ID,CNT_CHILDREN,AMT_INCOME_TOTAL,CNT_FAM_MEMBERS,CODE_GENDER_M,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_Y,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,...,OCCUPATION_TYPE_Private service staff,OCCUPATION_TYPE_Realty agents,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Secretaries,OCCUPATION_TYPE_Security staff,OCCUPATION_TYPE_Waiters/barmen staff,AGE,EMPLOYMENT_PERIOD,Months_from_Today,STATUS_y
0,5008804,0,4.275,2.0,1,1,1,0,0,0,...,0,0,0,0,0,0,32.89,12.44,0,1
1,5008805,0,4.275,2.0,1,1,1,0,0,0,...,0,0,0,0,0,0,32.89,12.44,0,1
2,5008806,0,1.125,2.0,1,1,1,0,0,0,...,0,0,0,0,1,0,58.83,3.11,0,0
3,5008808,0,2.700,1.0,0,0,1,0,0,0,...,0,0,1,0,0,0,52.36,8.36,0,1
4,5008809,0,2.700,1.0,0,0,1,0,0,0,...,0,0,1,0,0,0,52.36,8.36,22,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434808,5149828,0,3.150,2.0,1,1,1,0,0,0,...,0,0,0,0,0,0,47.53,6.63,0,0
434809,5149834,0,1.575,2.0,0,0,1,0,0,0,...,0,0,0,0,0,0,33.94,3.63,0,0
434810,5149838,0,1.575,2.0,0,0,1,1,0,0,...,0,0,0,0,0,0,33.94,3.63,0,0
434811,5150049,0,2.835,2.0,0,0,1,0,0,0,...,0,0,1,0,0,0,49.20,1.79,0,0


# Machine Learning

In [40]:
# Initial imports.
import pandas as pd
from path import Path
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, balanced_accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC

## Split the Data into Training and Testing

In [41]:
# Create our features
X = credit_application_df.drop(columns="STATUS_y")

# Create our target
y = pd.DataFrame(credit_application_df["STATUS_y"])

In [42]:

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

Counter(y_train)

Counter({'STATUS_y': 1})

In [43]:
# Determine the shape of our training and testing sets.
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(27342, 45)
(9115, 45)
(27342, 1)
(9115, 1)


## Scale Data

In [44]:

# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [45]:
# Show the number of approved and not approved credit card applicants from credit_application_df
credit_application_df.groupby(["STATUS_y"]).count()["ID"].set_axis(["Rejected", "Approved"], axis='index')


Rejected    28113
Approved     8344
Name: ID, dtype: int64

# Class Imbalance
## As we can see from the results above, the existing classes in the dataset is not equally represented. This is referred to as Class Imbalance and can cause the machine learning models to be biased toward the majority class. In this case, the machine learning models will be better at predicting not approved applicants. Hence, to counter this problem, we will be using Oversampling, Undersampling and Combination sampling techniques.

# Random Oversampling

In [46]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler()
X_resampled, y_resampled = ros.fit_resample(X_train_scaled, y_train)

Counter(y_resampled)

Counter({'STATUS_y': 1})

## Logistic Regression

In [47]:
# Train the Logistic Regression model using the resampled data
log_model = LogisticRegression(solver='lbfgs', random_state=1)
log_model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [48]:
# Calculated the balanced accuracy score

predictions = log_model.predict(X_test_scaled)
acc_score = accuracy_score(y_test, predictions)

In [49]:
# Display the confusion matrix
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

In [50]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,3814,3265
Actual 1,1048,988


Accuracy Score : 0.5268239166209545
Classification Report
              precision    recall  f1-score   support

           0       0.78      0.54      0.64      7079
           1       0.23      0.49      0.31      2036

    accuracy                           0.53      9115
   macro avg       0.51      0.51      0.48      9115
weighted avg       0.66      0.53      0.57      9115



## SVM

In [51]:
# Instantiate a linear SVM model

# svc_model = SVC(kernel='linear')
# # Fit the data
# svc_model.fit(X_resampled, y_resampled)

In [52]:
# Generate confusion matrix

In [53]:
# # Print the imbalanced classification report


## Decision Tree

In [54]:
# Creating the decision tree classifier instance.
tree_model = tree.DecisionTreeClassifier()
# Fitting the model.
tree_model.fit(X_resampled, y_resampled)

DecisionTreeClassifier()

In [55]:
predictions = tree_model.predict(X_test_scaled)
acc_score = accuracy_score(y_test, predictions)

In [56]:
# Display the confusion matrix
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

In [57]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,5928,1151
Actual 1,1136,900


Accuracy Score : 0.7490948985189249
Classification Report
              precision    recall  f1-score   support

           0       0.84      0.84      0.84      7079
           1       0.44      0.44      0.44      2036

    accuracy                           0.75      9115
   macro avg       0.64      0.64      0.64      9115
weighted avg       0.75      0.75      0.75      9115



# Random Forest

In [58]:

# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=98) 

In [59]:
# Fitting the model
rf_model = rf_model.fit(X_resampled, y_resampled)


In [60]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)

In [61]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,6132,947
Actual 1,1112,924


In [62]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)

In [63]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,6132,947
Actual 1,1112,924


Accuracy Score : 0.774108612177729
Classification Report
              precision    recall  f1-score   support

           0       0.85      0.87      0.86      7079
           1       0.49      0.45      0.47      2036

    accuracy                           0.77      9115
   macro avg       0.67      0.66      0.66      9115
weighted avg       0.77      0.77      0.77      9115



In [64]:
# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.3425201314217091, 'ID'),
 (0.12996469352468046, 'AGE'),
 (0.09580625132547177, 'EMPLOYMENT_PERIOD'),
 (0.09535917331401998, 'AMT_INCOME_TOTAL'),
 (0.08885901028009752, 'Months_from_Today'),
 (0.021402379995524322, 'CNT_FAM_MEMBERS'),
 (0.016742189364947178, 'CNT_CHILDREN'),
 (0.016369027024751404, 'FLAG_OWN_CAR_Y'),
 (0.01586099385267252, 'FLAG_OWN_REALTY_Y'),
 (0.01570554315048325, 'CODE_GENDER_M'),
 (0.015014690133604602, 'NAME_INCOME_TYPE_Working'),
 (0.011314781621919182, 'NAME_FAMILY_STATUS_Married'),
 (0.009801430320968925, 'NAME_EDUCATION_TYPE_Secondary / secondary special'),
 (0.009482053679180537, 'OCCUPATION_TYPE_Laborers'),
 (0.009404458178159344, 'OCCUPATION_TYPE_No Occupation Type'),
 (0.009312180139083337, 'NAME_EDUCATION_TYPE_Higher education'),
 (0.0076711513549763575, 'OCCUPATION_TYPE_Core staff'),
 (0.007424039380280535, 'OCCUPATION_TYPE_Sales staff'),
 (0.0073299262907783125, 'NAME_FAMILY_STATUS_Single / not married'),
 (0.006856232856408063, 'OCCUPATION_TYPE_Man

## Gradient Boosted Tree

In [65]:
learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
   classifier = GradientBoostingClassifier(n_estimators=20,
   learning_rate=learning_rate,
   max_features=5,
   max_depth=3,
   random_state=0)
   classifier.fit(X_resampled, y_resampled)
   print("Learning rate: ", learning_rate)
   print("Accuracy score (training): {0:.3f}".format(
       classifier.score(
           X_train_scaled,
           y_train)))
   print("Accuracy score (validation): {0:.3f}".format(
       classifier.score(
           X_test_scaled,
           y_test)))

Learning rate:  0.05
Accuracy score (training): 0.541
Accuracy score (validation): 0.525
Learning rate:  0.1
Accuracy score (training): 0.554
Accuracy score (validation): 0.533
Learning rate:  0.25
Accuracy score (training): 0.575
Accuracy score (validation): 0.544
Learning rate:  0.5
Accuracy score (training): 0.580
Accuracy score (validation): 0.553
Learning rate:  0.75
Accuracy score (training): 0.576
Accuracy score (validation): 0.549
Learning rate:  1
Accuracy score (training): 0.596
Accuracy score (validation): 0.568


In [66]:
GB_classifier = GradientBoostingClassifier(n_estimators=20,
   learning_rate=0.75, max_features=5, max_depth=3, random_state=0)

GB_classifier.fit(X_resampled, y_resampled)
predictions = GB_classifier.predict(X_test_scaled)

In [67]:
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
   cm, index=["Actual 0", "Actual 1"],
   columns=["Predicted 0", "Predicted 1"]
)
acc_score = accuracy_score(y_test, predictions)

In [68]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,3914,3165
Actual 1,947,1089


Accuracy Score : 0.5488754799780582
Classification Report
              precision    recall  f1-score   support

           0       0.81      0.55      0.66      7079
           1       0.26      0.53      0.35      2036

    accuracy                           0.55      9115
   macro avg       0.53      0.54      0.50      9115
weighted avg       0.68      0.55      0.59      9115



# SMOTE Oversampling

In [69]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE

X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy='auto').fit_resample(X_train_scaled, y_train)
Counter(y_resampled)

Counter({'STATUS_y': 1})

## Logistic Regression

In [70]:
# Train the Logistic Regression model using the resampled data
log_model = LogisticRegression(solver='lbfgs', random_state=1)
log_model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [71]:
# Calculated the balanced accuracy score
predictions = log_model.predict(X_test_scaled)
acc_score = accuracy_score(y_test, predictions)

In [72]:
# Display the confusion matrix
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])


In [73]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,3702,3377
Actual 1,1025,1011


Accuracy Score : 0.5170597915523861
Classification Report
              precision    recall  f1-score   support

           0       0.78      0.52      0.63      7079
           1       0.23      0.50      0.31      2036

    accuracy                           0.52      9115
   macro avg       0.51      0.51      0.47      9115
weighted avg       0.66      0.52      0.56      9115



## Decision Tree

In [74]:
# Creating the decision tree classifier instance.
tree_model = tree.DecisionTreeClassifier()
# Fitting the model.
tree_model.fit(X_resampled, y_resampled)


DecisionTreeClassifier()

In [75]:
predictions = tree_model.predict(X_test_scaled)
acc_score = accuracy_score(y_test, predictions)


In [76]:
# Display the confusion matrix
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,5578,1501
Actual 1,1064,972


Accuracy Score : 0.7185957213384531
Classification Report
              precision    recall  f1-score   support

           0       0.84      0.79      0.81      7079
           1       0.39      0.48      0.43      2036

    accuracy                           0.72      9115
   macro avg       0.62      0.63      0.62      9115
weighted avg       0.74      0.72      0.73      9115



## Random Forest

In [77]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=98) 

In [78]:
# Fitting the model
rf_model = rf_model.fit(X_resampled, y_resampled)
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)

In [79]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)

In [80]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,6075,1004
Actual 1,1106,930


Accuracy Score : 0.7685134393856281
Classification Report
              precision    recall  f1-score   support

           0       0.85      0.86      0.85      7079
           1       0.48      0.46      0.47      2036

    accuracy                           0.77      9115
   macro avg       0.66      0.66      0.66      9115
weighted avg       0.76      0.77      0.77      9115



In [81]:
# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.2633776152408472, 'ID'),
 (0.1318451229171966, 'AGE'),
 (0.11770774864183764, 'AMT_INCOME_TOTAL'),
 (0.0948295304482635, 'EMPLOYMENT_PERIOD'),
 (0.07673628018579018, 'Months_from_Today'),
 (0.0285807760035264, 'CNT_FAM_MEMBERS'),
 (0.025187187661906776, 'CNT_CHILDREN'),
 (0.024449704329101936, 'FLAG_OWN_REALTY_Y'),
 (0.023962179065929005, 'FLAG_OWN_CAR_Y'),
 (0.02223835166328812, 'NAME_INCOME_TYPE_Working'),
 (0.0196569610243178, 'CODE_GENDER_M'),
 (0.014930701538675128, 'NAME_FAMILY_STATUS_Married'),
 (0.01314888858745846, 'NAME_EDUCATION_TYPE_Secondary / secondary special'),
 (0.012795932808113007, 'NAME_EDUCATION_TYPE_Higher education'),
 (0.011827300354708001, 'OCCUPATION_TYPE_No Occupation Type'),
 (0.010752182259540137, 'OCCUPATION_TYPE_Laborers'),
 (0.009061663165359975, 'OCCUPATION_TYPE_Core staff'),
 (0.008807149337493377, 'OCCUPATION_TYPE_Sales staff'),
 (0.007799444989753048, 'NAME_HOUSING_TYPE_House / apartment'),
 (0.007477422270475621, 'OCCUPATION_TYPE_Managers'),
 (0

## Gradient Boot Tree

In [82]:
learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
   classifier = GradientBoostingClassifier(n_estimators=20,
   learning_rate=learning_rate,
   max_features=5,
   max_depth=3,
   random_state=0)
   classifier.fit(X_resampled, y_resampled)
   print("Learning rate: ", learning_rate)
   print("Accuracy score (training): {0:.3f}".format(
       classifier.score(
           X_train_scaled,
           y_train)))
   print("Accuracy score (validation): {0:.3f}".format(
       classifier.score(
           X_test_scaled,
           y_test)))

Learning rate:  0.05
Accuracy score (training): 0.568
Accuracy score (validation): 0.558
Learning rate:  0.1
Accuracy score (training): 0.575
Accuracy score (validation): 0.560
Learning rate:  0.25
Accuracy score (training): 0.583
Accuracy score (validation): 0.574
Learning rate:  0.5
Accuracy score (training): 0.621
Accuracy score (validation): 0.611
Learning rate:  0.75
Accuracy score (training): 0.624
Accuracy score (validation): 0.610
Learning rate:  1
Accuracy score (training): 0.616
Accuracy score (validation): 0.607


In [83]:
GB_classifier = GradientBoostingClassifier(n_estimators=20,
   learning_rate=0.75, max_features=5, max_depth=3, random_state=0)

GB_classifier.fit(X_resampled, y_resampled)
predictions = GB_classifier.predict(X_test_scaled)

In [84]:
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
   cm, index=["Actual 0", "Actual 1"],
   columns=["Predicted 0", "Predicted 1"]
)
acc_score = accuracy_score(y_test, predictions)
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,4775,2304
Actual 1,1253,783


Accuracy Score : 0.6097641250685683
Classification Report
              precision    recall  f1-score   support

           0       0.79      0.67      0.73      7079
           1       0.25      0.38      0.31      2036

    accuracy                           0.61      9115
   macro avg       0.52      0.53      0.52      9115
weighted avg       0.67      0.61      0.63      9115

